# Resumen: Web Scraping de Reseñas desde IMDb con **Selenium** y **BeautifulSoup**

## Pasos
1. **Inicializar**:
   - Crear listas vacías (`reseñas_totales` y `peliculas_sin_reseñas`) para almacenar datos.
   - Configurar un contador para las películas sin reseñas.

2. **Automatización con Selenium**:
   - Iterar sobre los títulos de películas y buscar cada uno en el campo de búsqueda de IMDb.
   - Seleccionar el primer título sugerido y realizar la búsqueda.

3. **Extraer reseñas**:
   - Intentar localizar el botón de reseñas y hacer clic en él.
   - Obtener los artículos de reseñas disponibles y extraer su título y comentario.
   - Guardar las reseñas extraídas en `reseñas_totales`.

4. **Manejo de errores**:
   - Si no hay reseñas disponibles, registrar la película en `peliculas_sin_reseñas` con un mensaje indicando la ausencia de datos.
   - Manejar excepciones para errores de búsqueda o extracción.

5. **Convertir y guardar resultados**:
   - Crear un DataFrame de pandas con los datos recopilados.
   - Guardar el DataFrame en un archivo CSV.

#### Librerías

In [1]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

#### Títulos de las películas a scrapear

In [2]:
titles_data = pd.read_csv("C:\\Users\\Usuario\\Desktop\\Netflix\\Raw_data\\titulos_no_scrapeados.csv",index_col=0)
titles_data.rename(columns={'0': 'title'}, inplace=True)
titles_data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Usuario\\Desktop\\Netflix\\Raw_data\\titulos_no_scrapeados.csv'

In [3]:
titles_data

,title
0,Freej Al Taibeen
1,Ayotzinapa: The Turtle's Pace
2,What She Put on the Table
3,Learning Songs by Little Baby Bum: Nursery Rhy...
4,Joaquín Reyes: Una y no más
...,...
568,로보카폴리 쏭쏭뮤지엄
569,Thug Life
570,Dead Kids
571,Pawn Stars


In [4]:
# bucle para obtener los titulos de las películas a scrapear
titulos = []
for titles in titles_data["title"]:
    titulos.append(titles)

# lista de titulos a scrapear
titulos

['Freej Al Taibeen',
 "Ayotzinapa: The Turtle's Pace",
 'What She Put on the Table',
 'Learning Songs by Little Baby Bum: Nursery Rhyme Friends',
 'Joaquín Reyes: Una y no más',
 'The Insomnia Club',
 'The Hows of Us',
 'Out of Many, One',
 'Loo Loo Kids Johny & Friends Musical Adventure',
 'Rafinha Bastos: Ultimatum',
 'Todo lo que sería Lucas Lauriente',
 'Voulez-vous rire avec moi ce soir?',
 'Coco y Raulito: Carrusel de ternura',
 "Eve's Apple",
 'Agustín Aristarán: Soy Rada',
 "Al-Khawaga's Dilemma",
 'Hadithi za Kumekucha: Fatuma',
 'DTC –Yukemuri Junjo Hen– from HiGH&LOW',
 'Shiva VS Autobots',
 'Motu Patlu in Dragon World',
 'Super Monsters Save Halloween',
 'Na stojáka v kině',
 'Motu Patlu: Mission Moon',
 'Shiva: The Secret World Of Vedas City',
 'Mau Nieto: viviendo sobrio… desde el bar',
 "Mrs. Miracle Animals: An Squirrel's Princess Bride",
 'True: Wonderful Wishes',
 'Yoo Byung Jae: Discomfort Zone',
 'Catch.er',
 'Ricardo Quevedo: Hay gente así',
 'Jani Dueñas: Grandes 

### Rutas y navegador

In [5]:
# Ruta del GeckoDriver
driver_path = 'C:/Users/Usuario/Desktop/Master Data& IA/APIS y Web scraping/chromedriver firefoxdriver/geckodriver.exe'

# Ruta del ejecutable de Firefox
firefox_binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'

# Configurar las opciones de Firefox
firefox_options = Options()
firefox_options.binary_location = firefox_binary_path

# Inicializar el driver
service = Service(driver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)

# Cargar una página de prueba
driver.get("https://www.imdb.com/es-es/?ref_=nv_home")

# Cerrar el driver después de unos segundos
import time
time.sleep(1)

###  Aceptar Cookies

In [6]:
# Esperar que el botón de aceptar cookies esté visible y clicarlo
try:
    aceptar_cookies = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.icb-btn:nth-child(2)"))
    )
    aceptar_cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("El botón de cookies no se encontró o ya estaba aceptado.")

Cookies aceptadas.


### Lógica del scraping

In [7]:
# Lista para almacenar todas las reseñas de todas las películas
reseñas_totales = []
peliculas_sin_reseñas = []
contador = 0

for titulo_pelicula in titulos[300::]:
    print(f"Buscando el título: {titulo_pelicula}")
    
    try:
        # Esperar a que el campo de búsqueda esté presente
        campo_busqueda = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#suggestion-search"))
        )
        
        # Limpiar el campo y escribir el título
        campo_busqueda.clear()
        campo_busqueda.send_keys(titulo_pelicula)
        print(f"Título '{titulo_pelicula}' ingresado en el campo de búsqueda.")
        
        time.sleep(1.5)  # Pausa breve para permitir la selección
        campo_busqueda.send_keys(Keys.ARROW_DOWN)  # Seleccionar primer título sugerido
        time.sleep(1.5)  # Pausa breve
        campo_busqueda.send_keys(Keys.RETURN)  # Presionar "Enter"
        print("Primer título seleccionado y búsqueda ejecutada.")
        time.sleep(1.5)  # Pausa breve
        
        # Intentar encontrar el botón de reseñas
        try:
            boton_resenas = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".sc-1fdae676-0 > li:nth-child(2) > a:nth-child(1)"))
            )
            boton_resenas.click()
            print("Botón de reseñas clicado correctamente.")
            
            # Extraer las reseñas de la película actual
            articulos_resenas = driver.find_elements(By.CSS_SELECTOR, "article.sc-d59f276d-1")
            
            for articulo in articulos_resenas:
                try:
                    # Extraer título y comentario de la reseña
                    titulo_resena = articulo.find_element(By.CSS_SELECTOR, "a > h3").text.strip()
                    comentario_resena = "Comentario no disponible"
                    try:
                        comentario_resena = articulo.find_element(By.CSS_SELECTOR, "div[data-testid='review-overflow']").text.strip()
                    except Exception as e:
                        print(f"No se pudo extraer el comentario: {e}")
                    
                    # Almacenar los datos
                    reseñas_totales.append({
                        "title": titulo_pelicula,
                        "Título_commentario": titulo_resena,
                        "Comentario": comentario_resena
                    })
                except Exception as e:
                    print(f"Error al procesar una reseña: {e}")
            
            print(f"Reseñas extraídas correctamente para '{titulo_pelicula}'.")
        
        except Exception:
            # Si no se encuentra el botón de reseñas, añadir un mensaje indicando que no hay reseñas
            print(f"La película '{titulo_pelicula}' NO TIENE RESEÑAS.")
            reseñas_totales.append({
                "title": titulo_pelicula,
                "Título_commentario": "No disponible",
                "Comentario": "Esta película no tiene reseñas disponibles."
            })
            peliculas_sin_reseñas.append(titulo_pelicula)  
            contador += 1
    except Exception as e:
        print(f"¡¡¡¡¡ERROR AL PROCESAR UNA PELICULA!!!!!!!! '{titulo_pelicula}': {e}")

# Convertir los datos totales a un DataFrame
df_reseñas_totales = pd.DataFrame(reseñas_totales)

Buscando el título: Bheem vs Aliens
Título 'Bheem vs Aliens' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Bheem vs Aliens' NO TIENE RESEÑAS.
Buscando el título: Tango with Me
Título 'Tango with Me' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Tango with Me' NO TIENE RESEÑAS.
Buscando el título: Osuofia in London 2
Título 'Osuofia in London 2' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Osuofia in London 2' NO TIENE RESEÑAS.
Buscando el título: Inventing David Geffen
Título 'Inventing David Geffen' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película 'Inventing David Geffen' NO TIENE RESEÑAS.
Buscando el título: 26 Years
Título '26 Years' ingresado en el campo de búsqueda.
Primer título seleccionado y búsqueda ejecutada.
La película '26 Years' NO TIENE RESEÑAS.
Buscando el título: Darling, 

In [8]:
# Resumen final
print(f"\nProceso completado. Total de películas procesadas: {titulos}")
print(f"Total de películas sin reseñas: {contador}")
print("Películas sin reseñas:")
print(peliculas_sin_reseñas)


Proceso completado. Total de películas procesadas: ['Freej Al Taibeen', "Ayotzinapa: The Turtle's Pace", 'What She Put on the Table', 'Learning Songs by Little Baby Bum: Nursery Rhyme Friends', 'Joaquín Reyes: Una y no más', 'The Insomnia Club', 'The Hows of Us', 'Out of Many, One', 'Loo Loo Kids Johny & Friends Musical Adventure', 'Rafinha Bastos: Ultimatum', 'Todo lo que sería Lucas Lauriente', 'Voulez-vous rire avec moi ce soir?', 'Coco y Raulito: Carrusel de ternura', "Eve's Apple", 'Agustín Aristarán: Soy Rada', "Al-Khawaga's Dilemma", 'Hadithi za Kumekucha: Fatuma', 'DTC –Yukemuri Junjo Hen– from HiGH&LOW', 'Shiva VS Autobots', 'Motu Patlu in Dragon World', 'Super Monsters Save Halloween', 'Na stojáka v kině', 'Motu Patlu: Mission Moon', 'Shiva: The Secret World Of Vedas City', 'Mau Nieto: viviendo sobrio… desde el bar', "Mrs. Miracle Animals: An Squirrel's Princess Bride", 'True: Wonderful Wishes', 'Yoo Byung Jae: Discomfort Zone', 'Catch.er', 'Ricardo Quevedo: Hay gente así', 

In [13]:
df_reseñas_totales.tail()

,title,Título_commentario,Comentario
228,Dahmer - Monster: The Jeffrey Dahmer Story,No disponible,Esta película no tiene reseñas disponibles.
229,Hidden Passion,No disponible,Esta película no tiene reseñas disponibles.
230,"My Lover, My Killer",No disponible,Esta película no tiene reseñas disponibles.
231,Booha,No disponible,Esta película no tiene reseñas disponibles.
232,If Only,No disponible,Esta película no tiene reseñas disponibles.


In [10]:
df_reseñas_totales["Título_commentario"].duplicated().sum()

133

### Exportación CSV

In [11]:
# Guardar en CSV
df_reseñas_totales.to_csv("scrap_ultimo_2.csv", index=False, encoding="utf-8")